In [45]:
import tensorflow as tf
import zipfile,os,shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O rockpaperscissors.zip



--2020-06-30 14:55:45--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘rockpaperscissors.zip’

rockpaperscissors.z 100%[===================>] 307.92M  17.4MB/s    in 13s     

2020-06-30 14:55:58 (24.4 MB/s) - ‘rockpaperscissors.zip’ saved [322873683/322873683]



In [65]:
dataset='rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(dataset, 'r')
zip_ref.extractall('Submission/')
zip_ref.close()




In [ ]:
os.makedirs("Submission/Training/paper", exist_ok=True)
os.makedirs("Submission/Training/scissors", exist_ok=True)
os.makedirs("Submission/Training/rock", exist_ok=True)
os.makedirs("Submission/Validasi/paper", exist_ok=True)
os.makedirs("Submission/Validasi/scissors", exist_ok=True)
os.makedirs("Submission/Validasi/rock", exist_ok=True)
os.listdir('Submission/Training')

In [67]:
#perbaikan logika pindah file
data_utama = 'Submission/rockpaperscissors/'
train_dir = 'Submission/Training/'
validation = 'Submission/Validasi/'
list_move = ['paper','rock','scissors']

for nama in list_move : 
  data = os.listdir(data_utama+nama)
  jumlah_data=len(data)
  validasi_data =  (round(jumlah_data*0.2))#validasi 20%
  for nama_file,file in enumerate(data) :
     if nama_file <= validasi_data :
         shutil.move(data_utama+nama+'/'+file,validation+nama)
     else :
         shutil.move(data_utama+nama+'/'+file,train_dir +nama)
  

In [68]:
train_paper = os.path.join(train_dir,'paper')
train_rock = os.path.join(train_dir,'rock')
train_scissors=os.path.join(train_dir,'scissors')

val_paper=os.path.join(validation,'paper')
val_rock=os.path.join(validation,'rock')
val_scissors=os.path.join(validation,'scissors')



In [69]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [72]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(150, 150),  
        batch_size=4,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation, 
        target_size=(150, 150), 
        batch_size=4, 
        class_mode='categorical')

Found 1748 images belonging to 3 classes.
Found 440 images belonging to 3 classes.


In [71]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [62]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=50,  
      epochs=20,
      validation_data=validation_generator,
      validation_steps=5,  
      verbose=2)

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

uploaded = files.upload()

for file in uploaded.keys():
 
 
  path = file
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  result = np.argmax(classes)

  if result==0:
    print('paper')
  elif result==1:
    print('rock')
  else :
    print('scissors')

In [ ]:
shutil.rmtree('Submission') #untuk hapus dir (jangan di run jika tidak butuh)